## Setup

In [2]:
import io
import re
import string
import numpy as np
import tensorflow as tf
from tqdm import tqdm
from tensorflow.keras import layers

### Parameters

In [3]:
SEED = 42
AUTOTUNE = tf.data.AUTOTUNE

### Generate training data helper function

In [14]:
def generate_training_data(sequences, window_size, num_ns, vocab_size, seed):

    #Init
    context_class = tf.reshape(tf.constant(0, dtype="int64"), (1, 1))
    targets, contexts, labels = [], [], []
    sampling_table = tf.keras.preprocessing.sequence.make_sampling_table(vocab_size)

    #Iterate over all sequences (sentences) in the dataset.
    for sequence in tqdm(sequences):

        #Generate positive skip-gram pairs for a sequence (sentence).
        positive_skip_grams, _ = tf.keras.preprocessing.sequence.skipgrams(
              sequence,
              vocabulary_size=vocab_size,
              sampling_table=sampling_table,
              window_size=window_size,
              negative_samples=0)

        for target_word, context_word in positive_skip_grams:
            context_class = tf.expand_dims(tf.constant([context_word], dtype="int64"), 1)
            negative_sampling_candidates, _, _ = tf.random.log_uniform_candidate_sampler(
                true_classes=context_class,
                num_true=1,
                num_sampled=num_ns,
                unique=True,
                range_max=vocab_size,
                seed=seed,
                name="negative_sampling")
            
        #Build context and label vectors (for one target word)
        context = tf.concat([tf.squeeze(context_class,1), negative_sampling_candidates], 0)
        label = tf.constant([1] + [0]*num_ns, dtype="int64")

        #Append each element from the training example to global lists.
        targets.append(target_word)
        contexts.append(context)
        labels.append(label)

    return targets, contexts, labels

## Prepare training data for word2vec

In [15]:
text_ds = tf.data.TextLineDataset('../../data/books/principles_of_neuroscience.txt')
text_ds = text_ds.filter(lambda x: tf.cast(tf.strings.length(x), bool))

### Vectorize sentences from the corpus

In [16]:
def custom_standardization(input_data):
    lowercase = tf.strings.lower(input_data)
    return tf.strings.regex_replace(lowercase,'[%s]' % re.escape(string.punctuation), '')


#Define the vocabulary size and the number of words in a sequence.
vocab_size = 4096
sequence_length = 10
vectorize_layer = layers.TextVectorization(
    standardize=custom_standardization,
    max_tokens=vocab_size,
    output_mode='int',
    output_sequence_length=sequence_length)

### Call TextVectorization.adapt on the text dataset to create vocabulary.

In [17]:
vectorize_layer.adapt(text_ds.batch(1024))

2023-02-01 12:15:16.645951: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


### Have a look at vocab

In [18]:
inverse_vocab = vectorize_layer.get_vocabulary()
print(inverse_vocab[:20])

['', '[UNK]', 'the', 'of', 'in', 'and', 'a', 'to', 'is', 'that', 'are', 'by', 'with', 'from', 'as', 'for', 'neurons', 'or', 'this', 'an']


### Vectorize and obtain sequences from the dataset

In [19]:
text_vector_ds = text_ds.batch(1024).prefetch(AUTOTUNE).map(vectorize_layer).unbatch()
sequences = list(text_vector_ds.as_numpy_iterator())

### Generate training examples from sequences

In [20]:
context_class = tf.reshape(tf.constant(0, dtype="int64"), (1, 1))
targets, contexts, labels = generate_training_data(
    sequences=sequences,
    window_size=2,
    num_ns=4,
    vocab_size=vocab_size,
    seed=SEED)

targets = np.array(targets)
contexts = np.array(contexts)
labels = np.array(labels)

print(f"targets.shape: {targets.shape}")
print(f"contexts.shape: {contexts.shape}")
print(f"labels.shape: {labels.shape}")

100%|████████████████████████████████████| 33257/33257 [01:16<00:00, 433.71it/s]


targets.shape: (33257,)
contexts.shape: (33257, 5)
labels.shape: (33257, 5)


### Configure the dataset for performance

In [21]:
BATCH_SIZE = 1024
BUFFER_SIZE = 10000
dataset = tf.data.Dataset.from_tensor_slices(((targets, contexts), labels))
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
dataset = dataset.cache().prefetch(buffer_size=AUTOTUNE)
print(dataset)

<PrefetchDataset element_spec=((TensorSpec(shape=(1024,), dtype=tf.int64, name=None), TensorSpec(shape=(1024, 5), dtype=tf.int64, name=None)), TensorSpec(shape=(1024, 5), dtype=tf.int64, name=None))>


## Model and training

In [22]:
class Word2Vec(tf.keras.Model):
    
    def __init__(self, vocab_size, embedding_dim):
        super(Word2Vec, self).__init__()
        self.target_embedding = layers.Embedding(vocab_size,
                                                 embedding_dim,
                                                 input_length=1,
                                                 name="w2v_embedding")
        self.context_embedding = layers.Embedding(vocab_size,
                                                  embedding_dim,
                                                  input_length=num_ns+1)

    
    def call(self, pair):
        target, context = pair
        if len(target.shape) == 2:
            target = tf.squeeze(target, axis=1)
        word_emb = self.target_embedding(target)
        context_emb = self.context_embedding(context)
        dots = tf.einsum('be,bce->bc', word_emb, context_emb)
        return dots

### Define loss function and compile model


In [23]:
def custom_loss(x_logit, y_true):
    return tf.nn.sigmoid_cross_entropy_with_logits(logits=x_logit, labels=y_true)

### Compile Model

In [25]:
num_ns = 4
embedding_dim = 128
word2vec = Word2Vec(vocab_size, embedding_dim)
word2vec.compile(optimizer='adam',
                 loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
                 metrics=['accuracy'])

### Fit

In [26]:
word2vec.fit(dataset, epochs=100)

Epoch 1/100
 3/32 [=>............................] - ETA: 0s - loss: 1.6092 - accuracy: 0.1976 

2023-02-01 12:17:58.137339: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


32/32 [==============================] - 1s 22ms/step - loss: 1.6065 - accuracy: 0.2994
Epoch 2/100
32/32 [==============================] - 0s 13ms/step - loss: 1.5846 - accuracy: 0.7397
Epoch 3/100
32/32 [==============================] - 0s 13ms/step - loss: 1.5465 - accuracy: 0.8356
Epoch 4/100
32/32 [==============================] - 0s 13ms/step - loss: 1.4777 - accuracy: 0.8503
Epoch 5/100
32/32 [==============================] - 0s 13ms/step - loss: 1.3771 - accuracy: 0.8514
Epoch 6/100
32/32 [==============================] - 0s 13ms/step - loss: 1.2571 - accuracy: 0.8518
Epoch 7/100
32/32 [==============================] - 0s 13ms/step - loss: 1.1341 - accuracy: 0.8551
Epoch 8/100
32/32 [==============================] - 0s 13ms/step - loss: 1.0181 - accuracy: 0.8617
Epoch 9/100
32/32 [==============================] - 0s 13ms/step - loss: 0.9125 - accuracy: 0.8725
Epoch 10/100
32/32 [==============================] - 0s 13ms/step - loss: 0.8178 - accuracy: 0.8829
Epoch 11/10

32/32 [==============================] - 0s 13ms/step - loss: 0.0531 - accuracy: 0.9919
Epoch 83/100
32/32 [==============================] - 0s 13ms/step - loss: 0.0527 - accuracy: 0.9919
Epoch 84/100
32/32 [==============================] - 0s 12ms/step - loss: 0.0524 - accuracy: 0.9919
Epoch 85/100
32/32 [==============================] - 0s 12ms/step - loss: 0.0522 - accuracy: 0.9919
Epoch 86/100
32/32 [==============================] - 0s 11ms/step - loss: 0.0519 - accuracy: 0.9919
Epoch 87/100
32/32 [==============================] - 0s 12ms/step - loss: 0.0516 - accuracy: 0.9919
Epoch 88/100
32/32 [==============================] - 0s 13ms/step - loss: 0.0513 - accuracy: 0.9919
Epoch 89/100
32/32 [==============================] - 0s 13ms/step - loss: 0.0511 - accuracy: 0.9918
Epoch 90/100
32/32 [==============================] - 0s 12ms/step - loss: 0.0508 - accuracy: 0.9918
Epoch 91/100
32/32 [==============================] - 0s 12ms/step - loss: 0.0506 - accuracy: 0.9918
Epo

<!-- <img class="tfo-display-only-on-site" src="images/word2vec_tensorboard.png"/> -->

### Create and save the vectors and metadata files

In [29]:
weights = word2vec.get_layer('w2v_embedding').get_weights()[0]
vocab = vectorize_layer.get_vocabulary()
out_v = io.open('../../data/results/neuralscience_skipgram_vectors.tsv', 'w', encoding='utf-8')
out_m = io.open('../../data/results/neuralscience_skipgram_metadata.tsv', 'w', encoding='utf-8')
for index, word in enumerate(vocab):
    if index == 0:
        continue  
    vec = weights[index]
    out_v.write('\t'.join([str(x) for x in vec]) + "\n")
    out_m.write(word + "\n")
out_v.close()
out_m.close()